# Sea Ice Extent Timeseries
## **ECCO**

In [1]:
from dask_gateway import Gateway
g = Gateway()
cluster = g.connect(g.list_clusters()[0].name)
client = cluster.get_client()

In [2]:
client

Client Scheduler: gateway://traefik-gcp-uscentral1b-staging-dask-gateway.staging:80/staging.31b2fee6c187448cbe30012cb8b49b82 Dashboard: /services/dask-gateway/clusters/staging.31b2fee6c187448cbe30012cb8b49b82/status,Cluster Workers: 30 Cores: 60 Memory: 240.00 GiB


In [18]:
client.close()
cluster.close()

In [2]:
import xarray as xr
from matplotlib import pyplot as plt
import gcsfs
import dask
import dask.array as dsa
import numpy as np
import intake
import fsspec
import xmitgcm
from xhistogram.xarray import histogram
from xmitgcm.llcreader.llcmodel import faces_dataset_to_latlon
from fastjmd95 import jmd95numba

## Saving Data

In [3]:
ice = xr.open_dataarray('sea-ice-ecco')

In [4]:
ice = ice.to_dataset()

In [5]:
ice

<xarray.Dataset>
Dimensions:  (i: 95, j: 50, time: 288)
Coordinates:
  * time     (time) datetime64[ns] 1992-01-15 1992-02-13 ... 2015-12-14
    k        int64 ...
Dimensions without coordinates: i, j
Data variables:
    SIarea   (time, j, i) float32 ...

In [6]:
import json
with open('/home/jovyan/WMT-project/pangeo-181919-f141378dc975.json') as token_file:
    token = json.load(token_file)
gcfs_w_token = gcsfs.GCSFileSystem(token=token)

In [7]:
#uploading seaice 
gcsmap_seaice = gcfs_w_token.get_mapper('pangeo-tmp/stb2145/ECCO/datasets/seaice.zarr')
ice.to_zarr(gcsmap_seaice, mode='w')